<a href="https://colab.research.google.com/github/Oluwadara-Adepoju/Womxn/blob/main/Womxn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,  mean_squared_error, f1_score, roc_auc_score

from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

In [3]:
!pip install catboost==0.22 --quiet

     |████████████████████████████████| 64.4MB 102kB/s 


In [4]:
import os, sys, gc, warnings, random
import numpy as np 
import pandas as pd 
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import lightgbm as lgb  
from catboost import CatBoostClassifier ,Pool

from sklearn.metrics import auc, classification_report, roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [57]:
class CFG :
  SEED = 42
  n_splits = 10
  catboost_params = {'learning_rate':0.03,'iterations':10000,'eval_metric':'AUC',
                      'use_best_model' :True,'verbose':100,'random_seed': 0,
                      'devices':'0:1',}#'task_type':"GPU",}

  lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                'n_estimators': 5000,'sub_sample' : 0.5,'colsample_bytree' : 0.9,
                'seed': SEED,'silent':False,'early_stopping_rounds': 100,'num_leaves':120}
                
  xgb_params = {
              'learning_rate':0.1,  'max_depth':7,
              'n_estimators':100, 'n_jobs': -1,
              'objective':'binary:logistic', 'random_state':SEED, 'verbosity':100,'eval_metric':'auc'}

               
  remove_features = ['ID', 'gender', 'target']
  categ_features = ['race','dwelling','dwelling_type','province_code','metro_code','nationality','RTH','marital_st','Lang_inside','Lang_outside','Education','lw_work','lw_business','help_on_household','job_or_business','nature_of_work']
  TARGET_COL = 'target'

In [54]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)

In [7]:
train= pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/Train (2).csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Women/Test (1).csv")
vd = pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/VariableDefinitions.csv")
ss=pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/SampleSubmission (1).csv")

In [8]:
seed_everything(CFG.SEED)

In [9]:
train['gender'] = label.fit_transform(train['gender'])
train['race'] = label.fit_transform(train['race'])
train['dwelling'] = label.fit_transform(train['dwelling'])
train['dwelling_type'] = label.fit_transform(train['dwelling_type'])
train['province_code'] = label.fit_transform(train['province_code'])
train['metro_code'] = label.fit_transform(train['metro_code'])
train['nationality'] = label.fit_transform(train['nationality'])
train['RTH'] = label.fit_transform(train['RTH'])
train['marital_st'] = label.fit_transform(train['marital_st'])
train['Lang_inside'] = label.fit_transform(train['Lang_inside'])
train['Lang_outside'] = label.fit_transform(train['Lang_outside'])
train['Education'] = label.fit_transform(train['Education'])
train['lw_work'] = label.fit_transform(train['lw_work'])
train['lw_business'] = label.fit_transform(train['lw_business'])
train['help_on_household'] = label.fit_transform(train['help_on_household'])
train['job_or_business'] = label.fit_transform(train['job_or_business'])
train['nature_of_work'] = label.fit_transform(train['nature_of_work'])


In [10]:
test['gender'] = label.fit_transform(test['gender'])
test['race'] = label.fit_transform(test['race'])
test['dwelling'] = label.fit_transform(test['dwelling'])
test['dwelling_type'] = label.fit_transform(test['dwelling_type'])
test['province_code'] = label.fit_transform(test['province_code'])
test['metro_code'] = label.fit_transform(test['metro_code'])
test['nationality'] = label.fit_transform(test['nationality'])
test['RTH'] = label.fit_transform(test['RTH'])
test['marital_st'] = label.fit_transform(test['marital_st'])
test['Lang_inside'] = label.fit_transform(test['Lang_inside'])
test['Lang_outside'] = label.fit_transform(test['Lang_outside'])
test['Education'] = label.fit_transform(test['Education'])
test['lw_work'] = label.fit_transform(test['lw_work'])
test['lw_business'] = label.fit_transform(test['lw_business'])
test['help_on_household'] = label.fit_transform(test['help_on_household'])
test['job_or_business'] = label.fit_transform(test['job_or_business'])
test['nature_of_work'] = label.fit_transform(test['nature_of_work'])

In [11]:
features_columns = [col for col in train.columns if col not in CFG.remove_features]
len(features_columns)

18

In [55]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True, random_state=CFG.SEED)

X , y   = train[features_columns] , train[CFG.TARGET_COL]

oof_cat = np.zeros((train.shape[0],))
test['target'] = 0
cat_preds= []

for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print(50*'-')
    print('Fold:',fold_+1)
    X_train, y_train = X.iloc[trn_idx,:], y[trn_idx] 
    X_test, y_test = X.iloc[val_idx,:], y[val_idx] 
       
    estimator = CatBoostClassifier(**CFG.catboost_params)
    estimator.fit(Pool(X_train,y_train,cat_features = CFG.categ_features),
                  eval_set = Pool(X_test,y_test,cat_features = CFG.categ_features),
                  early_stopping_rounds=200)
    
    y_pred_val = estimator.predict_proba(X_test)[:,1]
    oof_cat[val_idx] = y_pred_val
    y_pred_test = estimator.predict_proba(test[features_columns])[:,1]
    cat_preds.append(y_pred_test)
    print(50*'-')
    print()
print('OOF score :',roc_auc_score(y, oof_cat))

--------------------------------------------------
Fold: 1
0:	test: 0.5747899	best: 0.5747899 (0)	total: 31.7ms	remaining: 5m 16s
100:	test: 0.6462535	best: 0.6466620 (97)	total: 1.76s	remaining: 2m 53s
200:	test: 0.6455182	best: 0.6512838 (142)	total: 4.64s	remaining: 3m 46s
300:	test: 0.6438375	best: 0.6512838 (142)	total: 7.71s	remaining: 4m 8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6512838469
bestIteration = 142

Shrink model to first 143 iterations.
--------------------------------------------------

--------------------------------------------------
Fold: 2
0:	test: 0.5908077	best: 0.5908077 (0)	total: 28.4ms	remaining: 4m 44s
100:	test: 0.6945256	best: 0.6980319 (95)	total: 1.74s	remaining: 2m 50s
200:	test: 0.7026993	best: 0.7029292 (199)	total: 4.66s	remaining: 3m 47s
300:	test: 0.7101947	best: 0.7145633 (290)	total: 7.82s	remaining: 4m 11s
400:	test: 0.7154255	best: 0.7158393 (395)	total: 11.1s	remaining: 4m 26s
500:	test: 0.7099533	best: 0.71590

In [56]:
catboost_preds = np.mean(cat_preds,axis=0)

In [58]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True, random_state=CFG.SEED)

X , y   = train[features_columns] , train[CFG.TARGET_COL]

oof_lgb = np.zeros((train.shape[0],))
test['target'] = 0
lgb_preds = []

for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print(50*'-')
    print('Fold:',fold_+1)

    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 
        
    train_data = lgb.Dataset(tr_x, label=tr_y,categorical_feature=CFG.categ_features)
    valid_data = lgb.Dataset(vl_x, label=vl_y,categorical_feature=CFG.categ_features)

    estimator = lgb.train(CFG.lgb_params,train_data,valid_sets = [train_data,valid_data],verbose_eval = 100)
    
    y_pred_val = estimator.predict(vl_x,num_iteration=estimator.best_iteration)
    oof_lgb[val_idx] = y_pred_val
    
    y_pred_test = estimator.predict(test[features_columns],num_iteration=estimator.best_iteration)
    lgb_preds.append(y_pred_test)
    print(50*'-')

print('OOF score :',roc_auc_score(y, oof_lgb))

--------------------------------------------------
Fold: 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.999995	valid_1's auc: 0.646639
Early stopping, best iteration is:
[27]	training's auc: 0.989368	valid_1's auc: 0.65852
--------------------------------------------------
--------------------------------------------------
Fold: 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.999993	valid_1's auc: 0.737291
Early stopping, best iteration is:
[54]	training's auc: 0.999513	valid_1's auc: 0.742786
--------------------------------------------------
--------------------------------------------------
Fold: 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.999996	valid_1's auc: 0.62456
Early stopping, best iteration is:
[28]	training's auc: 0.990301	valid_1's auc: 0.651898
--------------------------------------------------
--------------------------------------------------


In [59]:
from xgboost import XGBClassifier
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True, random_state=CFG.SEED)

X , y   = train[features_columns] , train[CFG.TARGET_COL]

oof_xgb = np.zeros((train.shape[0],))
test['target'] = 0
xgb_preds = []

for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print(50*'-')
    print('Fold:',fold_+1)

    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 

    xgb=XGBClassifier(learning_rate=0.01,subsample=0.7,colsample_bytree=0.9,reg_alpha=10,
               n_jobs=-1,n_estimators=5000,max_depth= 5,random_state=34)
    estimator = xgb.fit(tr_x, tr_y, early_stopping_rounds = 200, eval_metric="auc",
                           eval_set=[(vl_x, vl_y)],verbose=250)
    train_predict = estimator.predict_proba(tr_x, ntree_limit = estimator.get_booster().best_ntree_limit)[:,1]
    
  
        
  
    y_pred_val = estimator.predict_proba(vl_x, ntree_limit = estimator.get_booster().best_ntree_limit)[:,1]
    oof_xgb[val_idx] = y_pred_val
    
    y_pred_test = estimator.predict(test[features_columns], ntree_limit = estimator.get_booster().best_ntree_limit)
    xgb_preds.append(y_pred_test)
    print(50*'-')

print('OOF score :',roc_auc_score(y, oof_xgb))

--------------------------------------------------
Fold: 1
[0]	validation_0-auc:0.5
Will train until validation_0-auc hasn't improved in 200 rounds.
[250]	validation_0-auc:0.590558
[500]	validation_0-auc:0.625163
[750]	validation_0-auc:0.641573
[1000]	validation_0-auc:0.648366
[1250]	validation_0-auc:0.652194
[1500]	validation_0-auc:0.654015
[1750]	validation_0-auc:0.654902
[2000]	validation_0-auc:0.658193
[2250]	validation_0-auc:0.659781
[2500]	validation_0-auc:0.659314
Stopping. Best iteration:
[2336]	validation_0-auc:0.660294

--------------------------------------------------
--------------------------------------------------
Fold: 2
[0]	validation_0-auc:0.5
Will train until validation_0-auc hasn't improved in 200 rounds.
[250]	validation_0-auc:0.646391
[500]	validation_0-auc:0.666464
[750]	validation_0-auc:0.680822
[1000]	validation_0-auc:0.69356
[1250]	validation_0-auc:0.703883
[1500]	validation_0-auc:0.707493
[1750]	validation_0-auc:0.712597
[2000]	validation_0-auc:0.715839
[225

In [60]:
xgbm_preds = np.mean(xgb_preds,axis=0)

In [61]:
lightgbm_preds = np.mean(lgb_preds,axis=0)

In [19]:
test['target'] = catboost_preds
submission = test[['ID', 'target']]
submission.to_csv('Women-Solution_cat2.csv',index = False)

In [ ]:
test['target'] = lightgbm_preds
submission = test[['ID', 'target']]
submission.to_csv('Women-Solution_lgbm.csv',index = False)

In [66]:
test['target'] = xgbm_preds*0.6 +  lightgbm_preds*0.2 + catboost_preds*0.2
submission = test[['ID', 'target']]
submission.to_csv('Women-Solution_xgb_lgbm_cat_4.csv',index = False)

In [67]:
from google.colab import files
files.download('Women-Solution_xgb_lgbm_cat_4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>